# Claude Tools - Nexus-Style Integration

This notebook uses the ngrok + Colab Secrets approach from the Nexus project.

## Setup Instructions

1. **Add your secret to Colab**
   - Click the key icon (🔑) in the left sidebar
   - Add secret name: `sun_colab` (or your preferred name)
   - Add a secure random string as the value

2. **Run this notebook** to start the server


In [ ]:
# Install required packages
!pip install fastapi uvicorn pyngrok requests

In [ ]:
# Import libraries
import os
import json
import asyncio
from datetime import datetime
from typing import Dict, Any

from fastapi import FastAPI, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from google.colab import userdata
import uvicorn
from pyngrok import ngrok
import threading

# Get API key from Colab secrets
try:
    API_KEY = userdata.get('sun_colab')
    print(f"✅ Retrieved API key from secret")
    print(f"🔑 Key preview: {API_KEY[:10]}...")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Using default key for testing")
    API_KEY = "test-key-12345"

In [ ]:
# Create FastAPI app
app = FastAPI(title="Claude Tools Colab Server")

# Add CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Request models
class CodeRequest(BaseModel):
    code: str
    api_key: str
    timeout: int = 30

class PromptRequest(BaseModel):
    prompt: str
    type: str = "standard"
    api_key: str

# API key validation
def verify_api_key(api_key: str):
    if api_key != API_KEY:
        raise HTTPException(status_code=401, detail="Invalid API key")
    return True

@app.get("/")
async def root():
    return {
        "message": "Claude Tools Colab Server",
        "status": "running",
        "timestamp": datetime.now().isoformat()
    }

@app.get("/health")
async def health():
    return {"status": "healthy", "server": "claude-tools-colab"}

@app.post("/execute")
async def execute_code(request: CodeRequest):
    """Execute Python code"""
    verify_api_key(request.api_key)
    
    try:
        # Create a namespace for execution
        namespace = {}
        
        # Capture output
        from io import StringIO
        import sys
        
        old_stdout = sys.stdout
        sys.stdout = StringIO()
        
        try:
            exec(request.code, namespace)
            output = sys.stdout.getvalue()
            
            return {
                "success": True,
                "output": output,
                "timestamp": datetime.now().isoformat()
            }
        finally:
            sys.stdout = old_stdout
            
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "type": type(e).__name__
        }

@app.post("/api/optimize")
async def optimize_prompt(request: PromptRequest):
    """Optimize prompts (placeholder)"""
    verify_api_key(request.api_key)
    
    # Basic optimization logic
    optimized = request.prompt
    
    if request.type == "elaborate":
        optimized = f"A detailed and cinematic view of {request.prompt}, " \
                   f"with rich textures and dramatic lighting"
    
    return {
        "success": True,
        "original": request.prompt,
        "optimized": optimized,
        "type": request.type
    }

@app.get("/status")
async def status():
    """Get server status"""
    import psutil
    
    return {
        "status": "running",
        "memory_usage": psutil.virtual_memory().percent,
        "cpu_usage": psutil.cpu_percent(),
        "timestamp": datetime.now().isoformat()
    }

print("✅ FastAPI app created")

In [ ]:
# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print(f"🌐 Public URL: {ngrok_tunnel.public_url}")
print(f"\n📋 Save this URL to use with Claude Tools!")
print(f"\n🔗 Test your connection:")
print(f"   {ngrok_tunnel.public_url}/health")

In [ ]:
# Run the server
print("🚀 Starting Colab server...")
print("\n⚡ Server is running!")
print(f"\n📱 Use this in Claude Tools:")
print(f'bridge.set_colab_url("{ngrok_tunnel.public_url}")')
print(f"\n🛑 Press 'Stop' to shutdown the server")

# Run in a thread to not block the notebook
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=run_server)
server_thread.start()

## Testing the Server

Now your Colab server is running! Test it from Claude Tools:

```python
from colab_integration.nexus_bridge import NexusColabBridge

# Initialize bridge
bridge = NexusColabBridge({
    'api_key': 'your-secret-key'  # Same as in Colab secrets
})

# Set the ngrok URL (from output above)
bridge.set_colab_url("https://your-ngrok-url.ngrok.io")

# Test connection
bridge.test_connection()

# Execute code
result = bridge.execute_code("""
import numpy as np
print(f"NumPy version: {np.__version__}")
print("Hello from Colab!")
""")

print(result)
```
